In [1]:
# Approach:


'''
We used an approach where we are calculating similarities among both column names and data frame rows. The detailed explanation is as follows:

1. Load the dataset (no crucial pre-processing is needed as the input data provided is already in required format)
2. Create embeddings for all column names to capture their semantic relations with user queries
3. Create embeddigns for user query
4. Calculate similarities between these column and query embeddings
5. This will give us target columns in which the query needs to be searched by the LLM later during LLM inferencing.
6. Now create row descriptions for all rows: descriptions basically include all rows data for only target columns (refer to cell: )
7. Create embeddings for these row descriptions to find semantic relations with queries (the good part is, these embeddings only include target columns to be searched, hence improving LLM accuracy and speed)
8. Finally, calculate similarities with row embeddings and we have most relevant rows given that they had only relevant columns data in them
9. The row and column data will be provided to LLM now as context along with the query so it can directly give the exact required answer.

- this is a robust, effective, fast, and scalable approach as it only targets top relevant rows and columns, hence decreasing input size significantly for LLM and imporving its efficiency and speed of the whole system.
'''

'\nWe used an approach where we are calculating similarities among both column names and data frame rows. The detailed explanation is as follows:\n\n1. Load the dataset (no crucial pre-processing is needed as the input data provided is already in required format)\n2. Create embeddings for all column names to capture their semantic relations with user queries\n3. Create embeddigns for user query\n4. Calculate similarities between these column and query embeddings\n5. This will give us target columns in which the query needs to be searched by the LLM later during LLM inferencing.\n6. Now create row descriptions for all rows: descriptions basically include all rows data for only target columns (refer to cell: )\n7. Create embeddings for these row descriptions to find semantic relations with queries (the good part is, these embeddings only include target columns to be searched, hence improving LLM accuracy and speed)\n8. Finally, calculate similarities with row embeddings and we have most 

In [2]:
import numpy as np
import pandas as pd

In [6]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [8]:
input_df = pd.read_csv("input_table.csv")
qa_df = pd.read_excel("QA_dataset_share.xlsx")

In [9]:
input_df.head(10)

,Invoice ID,City,Gender,Product line,Unit price,Quantity,Total,Date,Payment,gross income,Rating
0,750-67-8428,Yangon,Female,Health and beauty,74.69,7,548.9715,1/5/2019,Ewallet,26.1415,9.1
1,226-31-3081,Naypyitaw,Female,Electronic accessories,15.28,5,80.2200,3/8/2019,Cash,3.8200,9.6
2,631-41-3108,Yangon,Male,Home and lifestyle,46.33,7,340.5255,3/3/2019,Credit card,16.2155,7.4
3,123-19-1176,Yangon,Male,Health and beauty,58.22,8,489.0480,1/27/2019,Ewallet,23.2880,8.4
4,373-73-7910,Yangon,Male,Sports and travel,86.31,7,634.3785,2/8/2019,Ewallet,30.2085,5.3
5,699-14-3026,Naypyitaw,Male,Electronic accessories,85.39,7,627.6165,3/25/2019,Ewallet,29.8865,4.1
6,355-53-5943,Yangon,Female,Electronic accessories,68.84,6,433.6920,2/25/2019,Ewallet,20.6520,5.8
7,315-22-5665,Naypyitaw,Female,Home and lifestyle,73.56,10,772.3800,2/24/2019,Ewallet,36.7800,8.0
8,665-32-9167,Yangon,Female,Health and beauty,36.26,2,76.1460,1/10/2019,Credit card,3.6260,7.2
9,692-92-5582,Mandalay,Female,Food and beverages,54.84,3,172.7460,2/20/2019,Credit card,8.2260,5.9


In [10]:
qa_df.head()

,question,row index,column index,answer,filtered row index,filtered column index,generated response
0,What product line is in the latest entry?,999,3,Fashion accessories,NaN,NaN,NaN
1,On what date did the first transaction occur?,"17, 245, 450, 484, 496, 523, 567, 696, 829, 83...",7,1/1/2019,NaN,NaN,NaN
2,What is the latest transaction date?,"158, 306, 473, 474, 643, 646, 671, 881, 883, 9...",7,3/30/2019,NaN,NaN,NaN
3,what is the max rating given in home and lifes...,"2, 7, 19, 22, 25, 39, 40, 41, 54, 56, 58, 61, ...","3,10",9.9,NaN,NaN,NaN
4,How many transactions involved Male customers ...,"331, 464, 540, 708, 710","2,10",5,NaN,NaN,NaN


In [11]:
input_df.columns

Index(['Invoice ID', 'City', 'Gender', 'Product line', 'Unit price',
       'Quantity', 'Total', 'Date', 'Payment', 'gross income', 'Rating'],
      dtype='object')

In [12]:
print(min(input_df['Unit price']))

10.08


In [16]:
!pip install tf-keras

DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  You can safely remove it manually.


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 1.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.7 MB 1.9 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/390.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/390.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/390.2 MB 1.7 MB/s eta 0:03:52
   ---------------------------------------- 1.0/390.2 MB 1.7 MB/s eta 0:03:45
   ---------------------------------------- 1.3/390.2 MB 1.8 MB/s eta 0:03:41
   ---------------------------------------- 1.8/390.2 MB 1.8 MB/s eta 0:03:37
   -----------------------

In [17]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [14]:
# Initialize the pre-trained Sentence-BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

columns = ['Invoice ID', 'City', 'Gender', 'Product line', 'Unit price',
           'Quantity', 'Total', 'Date', 'Payment', 'gross income', 'Rating']

column_embeddings = model.encode(columns)

NameError: name 'SentenceTransformer' is not defined

In [ ]:
def column_index_extractor(query):
  query_embedding = model.encode([query])

  similarities = cosine_similarity(query_embedding, column_embeddings)

  relevant_columns_indices = np.argsort(similarities[0])[::-1]

  relevant_columns = [columns[i] for i in relevant_columns_indices[:]]  # Top 2 relevant columns
  relevant_columns_indices = relevant_columns_indices[:2]
  return relevant_columns_indices


In [ ]:
query = "what is the max rating given in home and lifestyle?"
row_numbers = column_index_extractor(query)

In [ ]:
def create_filtered_row_description(row, relevant_columns):
    description = []
    for col in relevant_columns:
        description.append(f"{col}: {row[col]}")
    return ', '.join(description)

In [ ]:
def extract_relevant_rows(query):

    relevant_columns_indices = column_index_extractor(query)
    relevant_columns = input_df.columns[relevant_columns_indices]

    row_descriptions = input_df.apply(lambda row: create_filtered_row_description(row, relevant_columns), axis=1).tolist()

    row_embeddings = model.encode(row_descriptions)

    query_embedding = model.encode([query])

    similarities = cosine_similarity(query_embedding, row_embeddings)

    top_indices = similarities[0].argsort()[::-1][:]

    return top_indices

In [ ]:
row_numbers = extract_relevant_rows(query)

[429 575 979 345 971 570 789 258  78 156 760 203  43 129 450 652  14 420
 486 883]


In [ ]:
"2, 7, 19, 22, 25, 39, 40, 41, 54, 56, 58, 61, 74, 90, 99, 113, 114, 119, 123, 125, 137, 144, 148, 157, 166, 175, 186, 187, 188, 189, 190, 193, 197, 204, 207, 212, 215, 229, 243, 244, 245, 253, 254, 257, 266, 268, 269, 272, 273, 276, 280, 281, 286, 289, 297, 298, 299, 307, 324, 330, 347, 353, 363, 367, 372, 374, 376, 397, 401, 402, 408, 414, 416, 429, 437, 442, 470, 483, 488, 489, 493, 502, 509, 511, 517, 518, 521, 522, 534, 535, 537, 540, 545, 555, 559, 570, 591, 599, 605, 622, 628, 633, 645, 652, 685, 687, 695, 697, 699, 704, 711, 722, 726, 729, 736, 739, 740, 743, 747, 748, 753, 757, 765, 769, 783, 792, 796, 799, 805, 821, 834, 857, 860, 862, 863, 874, 875, 883, 901, 905, 909, 912, 918, 920, 921, 922, 930, 934, 935, 942, 948, 951, 954, 958, 966, 967, 971, 973, 996, 998"

In [ ]:
def extract_rel_row_col(query):
  rel_row = extract_relevant_rows(query)
  rel_col = column_index_extractor(query)
  return rel_row, rel_col

In [ ]:
query = "what is the max rating given in home and lifestyle?"

In [ ]:
extract_rel_row_col(query)

(array([429, 575, 979, 345, 971, 570, 789, 258,  78, 156, 760, 203,  43,
        129, 450, 652,  14, 420, 486, 883]),
 array([10,  9]))

In [ ]:
def extract_relevant_data(input_df, query):
    rel_row, rel_col = extract_rel_row_col(query)

    # Fetch relevant rows and columns
    relevant_data = input_df.iloc[rel_row, rel_col]
    return relevant_data

In [ ]:
rel_data = extract_relevant_data(input_df, query)

In [ ]:
rel_data.head(10)

,Rating,gross income
429,7.3,45.3250
575,6.3,28.4585
979,6.5,3.3885
345,7.6,35.0685
971,6.7,12.9185
570,7.6,20.5100
789,9.7,20.6185
258,6.5,6.4500
78,6.6,39.1550
156,6.1,3.6085


In [ ]:
from transformers import pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def fetch_extracted_col(df):
  filtered_col = []
  for _, row in df.iterrows():
    query = row['question']
    rel_col = column_index_extractor(query)
    filtered_col.append(rel_col)
  return filtered_col

In [ ]:
filtered_column = fetch_extracted_col(qa_df)

In [ ]:
def fetch_extracted_row(df):
  filtered_row = []
  for _, row in df.iterrows():
    query = row['question']
    rel_row = extract_relevant_rows(query)
    filtered_row.append(rel_row)
  return filtered_row

In [ ]:
filtered_rows = fetch_extracted_row(qa_df)

In [ ]:
def fill_filtered_indices(df):
    filtered_rows = []
    filtered_columns = []

    for _, row in df.iterrows():
        query = row['question']
        rel_rows, rel_cols = extract_rel_row_col(query)

        filtered_rows.append(', '.join(map(str, rel_rows)))
        filtered_columns.append(', '.join(map(str, rel_cols)))

    df['filtered row index'] = filtered_rows
    df['filtered column index'] = filtered_columns

    return df

In [ ]:
testing_qa_df = qa_df.copy()

In [ ]:
testing_qa_df.head()

,question,row index,column index,answer,filtered row index,filtered column index,generated response
0,What product line is in the latest entry?,999,3,Fashion accessories,NaN,NaN,NaN
1,On what date did the first transaction occur?,"17, 245, 450, 484, 496, 523, 567, 696, 829, 83...",7,1/1/2019,NaN,NaN,NaN
2,What is the latest transaction date?,"158, 306, 473, 474, 643, 646, 671, 881, 883, 9...",7,3/30/2019,NaN,NaN,NaN
3,what is the max rating given in home and lifes...,"2, 7, 19, 22, 25, 39, 40, 41, 54, 56, 58, 61, ...","3,10",9.9,NaN,NaN,NaN
4,How many transactions involved Male customers ...,"331, 464, 540, 708, 710","2,10",5,NaN,NaN,NaN


In [ ]:
testing_qa_df = fill_filtered_indices(testing_qa_df)

In [ ]:
testing_qa_df.head()

,question,row index,column index,answer,filtered row index,filtered column index,generated response
0,What product line is in the latest entry?,999,3,Fashion accessories,"370, 919, 136, 210, 399, 419, 553, 238, 156, 8...","3, 4",NaN
1,On what date did the first transaction occur?,"17, 245, 450, 484, 496, 523, 567, 696, 829, 83...",7,1/1/2019,"498, 426, 293, 707, 363, 956, 32, 317, 373, 65...","7, 8",NaN
2,What is the latest transaction date?,"158, 306, 473, 474, 643, 646, 671, 881, 883, 9...",7,3/30/2019,"366, 588, 560, 513, 627, 705, 14, 296, 508, 26...","7, 8",NaN
3,what is the max rating given in home and lifes...,"2, 7, 19, 22, 25, 39, 40, 41, 54, 56, 58, 61, ...","3,10",9.9,"429, 575, 979, 345, 971, 570, 789, 258, 78, 15...","10, 9",NaN
4,How many transactions involved Male customers ...,"331, 464, 540, 708, 710","2,10",5,"410, 273, 721, 858, 348, 32, 625, 365, 361, 28...","8, 10",NaN


In [ ]:
testing_qa_df.to_csv("answer_qa_df.csv", index=False)